In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string

from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from tqdm import tqdm


In [2]:
data1 = pd.read_csv("Datasets of Stocks Market and Stock News Articles\Datasets of Stocks Market and Stock News Articles\Stocks News Articles Datasets\Headlines_2015.csv")
data2 = pd.read_csv("Datasets of Stocks Market and Stock News Articles\Datasets of Stocks Market and Stock News Articles\Stocks News Articles Datasets\Headlines_2016.csv")
data3 = pd.read_csv("Datasets of Stocks Market and Stock News Articles\Datasets of Stocks Market and Stock News Articles\Stocks News Articles Datasets\Headlines_2017.csv")
data4 = pd.read_csv("Datasets of Stocks Market and Stock News Articles\Datasets of Stocks Market and Stock News Articles\Stocks News Articles Datasets\Headlines_2018.csv")
data5 = pd.read_csv("Datasets of Stocks Market and Stock News Articles\Datasets of Stocks Market and Stock News Articles\Stocks News Articles Datasets\Headlines_2019.csv")
data6 = pd.read_csv("Datasets of Stocks Market and Stock News Articles\Datasets of Stocks Market and Stock News Articles\Stocks News Articles Datasets\Headlines_2020.csv")
data7 = pd.read_csv("Datasets of Stocks Market and Stock News Articles\Datasets of Stocks Market and Stock News Articles\Stocks News Articles Datasets\Headlines_2021.csv")

df = pd.concat([data1, data2, data3, data4, data5, data6, data7])
print(len(data1), len(data2), len(data3),len(data4), len(data5), len(data6),len(data7))
print("Total Rows in dataframe:", len(df))

365 366 365 365 364 366 174
Total Rows in dataframe: 2365


In [3]:
df.head()

,Date,Headlines
0,2015-01-01,What Can We Expect From Apple Inc. In 2015? 1 ...
1,2015-01-02,What to expect from Apple in 2015 beyond its s...
2,2015-01-03,Mountie: An inexpensive and innovative way to ...
3,2015-01-04,"Donald Yacktman on the Sources of Moats, His C..."
4,2015-01-05,"SIM-free iPhone 6, 6 Plus reportedly debuting ..."


In [4]:
df.describe()

,Date,Headlines
count,2365,2365
unique,2335,2332
top,2019-11-19,MasterCard is expanding its mobile wallet in S...
freq,2,3


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2365 entries, 0 to 173
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       2365 non-null   object
 1   Headlines  2365 non-null   object
dtypes: object(2)
memory usage: 55.4+ KB


In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df.shape

(2365, 2)

In [8]:
df.isnull().sum()

Date         0
Headlines    0
dtype: int64

In [9]:
# printing some random reviews
sent_0 = df['Headlines'].values[0]
print(sent_0)
print("="*50)

sent_1000 = df['Headlines'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = df['Headlines'].values[1500]
print(sent_1500)
print("="*50)

sent_2000 = df['Headlines'].values[2000]
print(sent_2000)
print("="*50)

What Can We Expect From Apple Inc. In 2015? 1 Reason I Might Be Wrong About the Apple Inc. Retina MacBook Air More Evidence Apple Inc.&quot;s First Quarter Will Be Huge Samsung: New smart TVs in 2015 to run on Tizen OS
Should You Buy Applied Materials (AMAT) Stock As It Hits New High? Time To Take A Bite Out Of Apple? Amazon.com Unveils New Echo Products: What You Need to Know UPDATE 1-Microsoft search engine Bing to focus on PC search market -CEO Apple working on fix for iPhone 8 &quot;crackling&quot; sound issue     - CNET Microsoft search engine Bing to focus on PC search market -CEO Wall Street ends flat after Yellen, tech shares bounce (VIDEO) Apple just released new information about how facial recognition on the iPhone X works Microsoft search engine Bing to focus on PC search market: CEO Microsoft search engine Bing to focus on PC search market: CEO Amazon&quot;s New Alexa Gadgets Are Aiming For Every Nook And Cranny Of Your Life Shareholders Accuse Google and Alphabet of Withh

In [10]:
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_1500 = re.sub(r"http\S+", "", sent_1500)
sent_2000 = re.sub(r"http\S+", "", sent_2000)

print(sent_0)

What Can We Expect From Apple Inc. In 2015? 1 Reason I Might Be Wrong About the Apple Inc. Retina MacBook Air More Evidence Apple Inc.&quot;s First Quarter Will Be Huge Samsung: New smart TVs in 2015 to run on Tizen OS


In [11]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_2000, 'lxml')
text = soup.get_text()
print(text)

What Can We Expect From Apple Inc. In 2015? 1 Reason I Might Be Wrong About the Apple Inc. Retina MacBook Air More Evidence Apple Inc."s First Quarter Will Be Huge Samsung: New smart TVs in 2015 to run on Tizen OS
Should You Buy Applied Materials (AMAT) Stock As It Hits New High? Time To Take A Bite Out Of Apple? Amazon.com Unveils New Echo Products: What You Need to Know UPDATE 1-Microsoft search engine Bing to focus on PC search market -CEO Apple working on fix for iPhone 8 "crackling" sound issue     - CNET Microsoft search engine Bing to focus on PC search market -CEO Wall Street ends flat after Yellen, tech shares bounce (VIDEO) Apple just released new information about how facial recognition on the iPhone X works Microsoft search engine Bing to focus on PC search market: CEO Microsoft search engine Bing to focus on PC search market: CEO Amazon"s New Alexa Gadgets Are Aiming For Every Nook And Cranny Of Your Life Shareholders Accuse Google and Alphabet of Withholding Gender Pay Ga

In [12]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [13]:
sent_1500 = decontracted(sent_1500)
print(sent_1500)
print("="*50)

Prepare For The Prescription Drug Price Battle Amazon&apos;s Bezos & Twitter winners of week; Apple Store, InstaCart losers Apple: Don&quot;t Stop Believin&quot; Apple Inc. (AAPL) Position Cut by Sterling Investment Management Inc. Mycio Wealth Partners LLC Sells 6,122 Shares of Apple Inc. (AAPL) Apple Inc. (AAPL) Shares Bought by CFO4Life Group LLC Apple fixes FaceTime bug, new iOS 12.1.4 update available; big bounty for teenager founding bug


In [14]:
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

What Can We Expect From Apple Inc. In   Reason I Might Be Wrong About the Apple Inc. Retina MacBook Air More Evidence Apple Inc.&quot;s First Quarter Will Be Huge Samsung: New smart TVs in  to run on Tizen OS


In [15]:
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

Prepare For The Prescription Drug Price Battle Amazon apos s Bezos Twitter winners of week Apple Store InstaCart losers Apple Don quot t Stop Believin quot Apple Inc AAPL Position Cut by Sterling Investment Management Inc Mycio Wealth Partners LLC Sells 6 122 Shares of Apple Inc AAPL Apple Inc AAPL Shares Bought by CFO4Life Group LLC Apple fixes FaceTime bug new iOS 12 1 4 update available big bounty for teenager founding bug


In [16]:

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [18]:
from nltk.corpus import stopwords
preprocessed_headlines = []
# tqdm is for printing the status bar
for sentance in tqdm(df['Headlines'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(token.lower() for token in nltk.word_tokenize(sentance) if token.lower() not in stopwords.words('english'))
    preprocessed_headlines.append(sentance.strip())

100%|██████████████████████████████████████████████████████████████████████████████| 2365/2365 [03:47<00:00, 10.37it/s]


In [ ]:
#df['Headlines'].values
df['Headlines'].values[1500]

In [ ]:
preprocessed_headlines[1500]

In [ ]:
#BoW
count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(preprocessed_headlines)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

final_counts = count_vect.transform(preprocessed_headlines)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

In [ ]:
#tfidf
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(preprocessed_headlines)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf_vect.transform(preprocessed_headlines)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

In [ ]:
i=0
list_of_sentance=[]
for sentance in preprocessed_reviews:
    list_of_sentance.append(sentance.split())

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '